In [18]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import os

from transformers import pipeline

import requests
import json
import openai

from medisure.ml_logic.data import clean_text, eda_pipeline
from medisure.ml_logic.FDI import get_tau_score


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/raynasser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/raynasser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/raynasser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
base_path = '/Users/raynasser/code/raynasser/MediSure/data/raw_data'
foodrug = '/Users/raynasser/code/raynasser/MediSure/data/raw_data/FooDru'
ddi = '/Users/raynasser/code/raynasser/MediSure/data/raw_data/drugs_allergic_interaction'

In [48]:
files = [f for f in os.listdir(base_path) if f.endswith('.csv')]
files

['medicine_dataset.csv',
 'PubChemLite_31Oct2020_exposomics.csv',
 'Allergen_Status_of_Food_Products.csv',
 'drug_relation_dfe.csv',
 'Medicine.csv']

In [49]:
medicine_dataset = pd.read_csv(os.path.join(base_path, 'medicine_dataset.csv'), index_col='id')
medicine_dataset

/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/1606911670.py:1: DtypeWarning: Columns (42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  medicine_dataset = pd.read_csv(os.path.join(base_path, 'medicine_dataset.csv'), index_col='id')


,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,sideEffect3,...,sideEffect41,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class
id,,,,,,,,,,,,,,,,,,,,,
1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,NaN,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN
2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,Diarrhea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides
3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,Upset stomach,...,NaN,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN
4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,Nausea,...,NaN,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN,NaN,NaN,Diphenylmethane Derivative,No,RESPIRATORY,H1 Antihistaminics (second Generation)
5,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,NaN,...,NaN,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248214,zestrain 100mg/325mg/250mg tablet mr,Aclotec MR 100mg/325mg/250mg Tablet,SAMONEC MR TABLET,Uniclof 100mg/325mg/250mg Tablet MR,Macito 100mg/325mg/250mg Tablet MR,Rumatin 100mg/325mg/250mg Tablet MR,Nausea,Vomiting,Heartburn,Stomach pain,...,NaN,Treatment of Muscular pain,NaN,NaN,NaN,NaN,NaN,No,PAIN ANALGESICS,NaN
248215,zoxinace 200mg tablet sr,Algic 200mg Tablet SR,Topnac 200mg Tablet SR,Kindac 200mg Tablet SR,Ultranac 200mg Tablet SR,Bignac 200mg Tablet SR,Dizziness,Indigestion,Nausea,Abdominal pain,...,NaN,Pain relief,NaN,NaN,NaN,NaN,Dichlorobenzenes Derivative,No,PAIN ANALGESICS,NSAID's- Non-Selective COX 1&2 Inhibitors (ace...
248216,zivex 25mg tablet,HD Zine 25mg Tablet,Hydrocas 25mg Tablet,Hyzox 25 Tablet,Hydil 25mg Tablet,Zyzine 25mg Tablet,Sedation,Nausea,Vomiting,Upset stomach,...,NaN,Treatment of Anxiety,Treatment of Skin conditions with inflammation...,NaN,NaN,NaN,Piperazine Derivative,No,RESPIRATORY,H1 Antihistaminics (First Generation)


In [50]:
eda_pipeline(medicine_dataset)

_________________________________


Columns:
Index(['name', 'substitute0', 'substitute1', 'substitute2', 'substitute3',
       'substitute4', 'sideEffect0', 'sideEffect1', 'sideEffect2',
       'sideEffect3', 'sideEffect4', 'sideEffect5', 'sideEffect6',
       'sideEffect7', 'sideEffect8', 'sideEffect9', 'sideEffect10',
       'sideEffect11', 'sideEffect12', 'sideEffect13', 'sideEffect14',
       'sideEffect15', 'sideEffect16', 'sideEffect17', 'sideEffect18',
       'sideEffect19', 'sideEffect20', 'sideEffect21', 'sideEffect22',
       'sideEffect23', 'sideEffect24', 'sideEffect25', 'sideEffect26',
       'sideEffect27', 'sideEffect28', 'sideEffect29', 'sideEffect30',
       'sideEffect31', 'sideEffect32', 'sideEffect33', 'sideEffect34',
       'sideEffect35', 'sideEffect36', 'sideEffect37', 'sideEffect38',
       'sideEffect39', 'sideEffect40', 'sideEffect41', 'use0', 'use1', 'use2',
       'use3', 'use4', 'Chemical Class', 'Habit Forming', 'Therapeutic Class',
       'Action Class'],

TypeError: readline.redisplay() takes no arguments (1 given)

In [7]:
def lower(text):
    try:
        return text.lower()
    except:
        return text





def summarize(text):
    pipe = pipeline('summarization', model= 'sshleifer/distilbart-xsum-12-6')

    summary = pipe(text)
    return summary[0]['summary_text']
    # return pipe(text)





def clasic(text):

    pipe = pipeline('question-answering', model= 'deepset/roberta-base-squad2')

    return pipe(question = 'Decribe the interaction between food and the drug here?',context=text)

In [8]:
files = [f for f in os.listdir(foodrug) if f.endswith('.csv')]
files

['unique_foods.csv',
 'misc_sample.csv',
 'nodes.csv',
 'texts.csv',
 'unique_drugs.csv',
 'sample.csv',
 'cmap.csv',
 'cmap_foodrugs.csv',
 'misc_study.csv',
 'TM_interactions.csv',
 'study.csv',
 'topTable.csv']

In [9]:
# df8 = pd.read_csv('/Users/raynasser/Downloads/DSproj/FooDru/TM_interactions.csv', index_col='Unnamed: 0')
# df8

foodrug_TM_interactions = pd.read_csv(os.path.join(foodrug, 'TM_interactions.csv'), index_col='Unnamed: 0')
# foodrug_TM_interactions.drop(columns= ['link', 'citation'], inplace=True)
foodrug_TM_interactions



foodrug_texts = pd.read_csv(os.path.join(foodrug, 'texts.csv'), index_col='Unnamed: 0')
foodrug_texts.drop(columns= ['link', 'citation'], inplace=True)
foodrug_texts

/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/2647824967.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  foodrug_TM_interactions = pd.read_csv(os.path.join(foodrug, 'TM_interactions.csv'), index_col='Unnamed: 0')


,texts_ID,document
0,0,Moderate Food Interaction||GENERALLY AVOID: G...
1,2,Major Food Interaction||GENERALLY AVOID: Alco...
2,7,Moderate Food Interaction||MONITOR: Coadminis...
3,11,Moderate Drug Interaction||GENERALLY AVOID: E...
4,13,Moderate Drug Interaction||GENERALLY AVOID: T...
...,...,...
8346,8441,The Use of a Continuous Glucose Monitoring Sys...
8347,8442,Glycemic Optimization On Discharge From the Em...
8348,8443,PANDA: PKU Amino Acid Evaluation Phenylketonur...
8349,8444,SNAP: Study Nutrients in Adult PKU Phenylketon...


In [10]:
fdi_pre = foodrug_texts.merge(foodrug_TM_interactions,on="texts_ID")#.drop(columns=["link","citation"])
fdi_pre

,texts_ID,document,TM_interactions_ID,start_index,end_index,food,drug
0,0,Moderate Food Interaction||GENERALLY AVOID: G...,0.0,0.0,116.0,Grapefruit,abemaciclib
1,0,Moderate Food Interaction||GENERALLY AVOID: G...,1.0,1943.0,2220.0,carbohydrate,abemaciclib
2,0,Moderate Food Interaction||GENERALLY AVOID: G...,2.0,2289.0,2392.0,grapefruit,abemaciclib
3,2,Major Food Interaction||GENERALLY AVOID: Alco...,3.0,385.0,641.0,grapefruit,fentanyl
4,2,Major Food Interaction||GENERALLY AVOID: Alco...,4.0,643.0,846.0,grapefruit,fentanyl
...,...,...,...,...,...,...,...
15021,8443,PANDA: PKU Amino Acid Evaluation Phenylketonur...,15064.0,0.0,268.0,phenylalanine,Phenylalanine
15022,8443,PANDA: PKU Amino Acid Evaluation Phenylketonur...,15065.0,269.0,434.0,diet,Phenylketonuria
15023,8444,SNAP: Study Nutrients in Adult PKU Phenylketon...,15066.0,0.0,270.0,phenylalanine,Phenylalanine
15024,8444,SNAP: Study Nutrients in Adult PKU Phenylketon...,15067.0,271.0,436.0,diet,Phenylketonuria


In [11]:
fdi_pre["food"] = fdi_pre.food.map(lower)
fdi_pre["drug"] = fdi_pre.drug.map(lower)

In [12]:
fdi_pre

,texts_ID,document,TM_interactions_ID,start_index,end_index,food,drug
0,0,Moderate Food Interaction||GENERALLY AVOID: G...,0.0,0.0,116.0,grapefruit,abemaciclib
1,0,Moderate Food Interaction||GENERALLY AVOID: G...,1.0,1943.0,2220.0,carbohydrate,abemaciclib
2,0,Moderate Food Interaction||GENERALLY AVOID: G...,2.0,2289.0,2392.0,grapefruit,abemaciclib
3,2,Major Food Interaction||GENERALLY AVOID: Alco...,3.0,385.0,641.0,grapefruit,fentanyl
4,2,Major Food Interaction||GENERALLY AVOID: Alco...,4.0,643.0,846.0,grapefruit,fentanyl
...,...,...,...,...,...,...,...
15021,8443,PANDA: PKU Amino Acid Evaluation Phenylketonur...,15064.0,0.0,268.0,phenylalanine,phenylalanine
15022,8443,PANDA: PKU Amino Acid Evaluation Phenylketonur...,15065.0,269.0,434.0,diet,phenylketonuria
15023,8444,SNAP: Study Nutrients in Adult PKU Phenylketon...,15066.0,0.0,270.0,phenylalanine,phenylalanine
15024,8444,SNAP: Study Nutrients in Adult PKU Phenylketon...,15067.0,271.0,436.0,diet,phenylketonuria


In [13]:
# summ_ =foodrug_texts.merge(foodrug_TM_interactions,on="texts_ID").tail(50).apply(lambda x: x["document"][int(x["start_index"]):int(x["end_index"])], axis=1).map(summarize)

summ_ =fdi_pre.head(5).apply(lambda x: x["document"][int(x["start_index"]):int(x["end_index"])], axis=1).map(summarize)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 62, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 62, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Hardware accelerator e.g. GPU is available in the environment, but

In [14]:
summ_

0     Grapefruit juice may increase the concentrati...
1     A high-calorie meal increases the activity of...
2     A new treatment for bowel cancer has been app...
3     A small quantity of grapefruit juice can be u...
4     A study has suggested grapefruit may be a pot...
dtype: object

In [15]:
# for i in range(0, 5):
#     print(summ_[i][0]["summary_text"])

In [16]:
fdi_pre["clean_text"] = fdi_pre.apply(lambda x: x["document"][int(x["start_index"]):int(x["end_index"])], axis=1)

In [17]:
# fdi_pre['summarized_text'] = summ_[i][0]["summary_text"]

fdi_combo = fdi_pre.groupby(by="texts_ID").agg({"food":lambda x: x.mode()[0],"drug":lambda x: x.mode()[0],"clean_text":lambda x: '. '.join(x)}).reset_index()

In [18]:
fdi_combo.head(5)

,texts_ID,food,drug,clean_text
0,0,grapefruit,abemaciclib,Moderate Food Interaction||GENERALLY AVOID: G...
1,2,grapefruit,fentanyl,GENERALLY AVOID: Consumption of grapefruit ju...
2,7,grapefruit,paclitaxel,Moderate Food Interaction||MONITOR: Coadminis...
3,11,ethanol,opioid analgesics,Moderate Drug Interaction||GENERALLY AVOID: E...
4,13,ethanol,dextromethorphan,Moderate Drug Interaction||GENERALLY AVOID: T...


In [19]:
fdi_combo.shape

(8272, 4)

In [20]:
# summ_ = fdi_combo.head(5).clean_text.map(summarize)

In [21]:
# for i in range(5):
#     print(summ_[i][0]["summary_text"])

In [22]:
# fdi_combo["summarize"] = fdi_combo.clean_text.map(summarize)

fdi_map = fdi_combo.head(5)
fdi_map["summarize"] = fdi_map.clean_text.map(summarize)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/2695138707.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [23]:
fdi_map.iloc[3]

texts_ID                                                     11
food                                                    ethanol
drug                                          opioid analgesics
clean_text    Moderate Drug Interaction||GENERALLY AVOID:  E...
summarize      Ethanol can be used as a treatment for painki...
Name: 3, dtype: object

In [24]:


def tau_score(text):

    # Define the API key and endpoint
    API_KEY = os.environ.get("API_KEY")


    # Set up headers for the API request
    client = openai.OpenAI(api_key=API_KEY)


    # run = client.beta.threads.runs.create(
    # thread_id="thread_1LsGTC2QCPTf1AU0S1fVMvZW",
    # assistant_id="asst_fjluyXIaT4Q7tNQ3nJrybazU",
    # additional_instructions=text
    # #extra_query=text
    # )


    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "assistant", "content": """You are tasked with elevating interaction between food and drugs fron a summary from a study document.
Analyze and identify the food and the drug in the text and the kind of relationship between the two of them(improving health, neutral, degrading health).
Return a tau score that will reflect the power of the interaction between the 2 elements and also if it's negative or positive
return the result through a json object, The response must be in the following JSON format:
            {
                "food": "",
                "drug": "",
                "relationship": "",
                "tau": 0
            }
            Return only the JSON object. """} ,
        {
            "role": "user",
            "content": text
        }
    ],
        response_format= { "type": "json_object" }
)

    return json.loads(completion.choices[0].message.content)

In [25]:
# fdi_map = tau_score() # take summarized text

fdi_map["tau"] = fdi_map.summarize.map(tau_score)

/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/2855111671.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdi_map["tau"] = fdi_map.summarize.map(tau_score)


In [26]:
fdi_map#.texts_ID.duplicated()

fdi_map.shape

(5, 6)

In [27]:
fdi_map["tau"]

0    {'food': 'high-calorie meal', 'drug': 'abemaci...
1    {'food': 'grapefruit juice', 'drug': 'fentanyl...
2    {'food': 'grapefruit juice', 'drug': 'docetaxe...
3    {'food': 'Ethanol', 'drug': 'Painkillers', 're...
4    {'food': 'ethanol', 'drug': 'dextromethorphan'...
Name: tau, dtype: object

In [28]:
# fdi_map["tau_score"] = fdi_map["tau"].map(lambda x:x["tau"])

In [29]:
fdi_map.tau.iloc[3]

{'food': 'Ethanol',
 'drug': 'Painkillers',
 'relationship': 'Improving health',
 'tau': 0.7}

In [30]:
fdi_map[['food_', 'drug_', 'relationship', 'tau_score']] = fdi_map['tau'].apply(pd.Series)


/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/3934944602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdi_map[['food_', 'drug_', 'relationship', 'tau_score']] = fdi_map['tau'].apply(pd.Series)
/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/3934944602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdi_map[['food_', 'drug_', 'relationship', 'tau_score']] = fdi_map['tau'].apply(pd.Series)
/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/393494460

In [31]:
# for index, row in fdi_map.iterrows():
#     print(f"Row {index}: Food={row['food']}, Drug={row['drug']}, Relationship={row['relationship']}, Tau={row['tau']}")


for index,row in fdi_map[["summarize","tau_score"]].iterrows():
    print(f"""TAU Score:{row["tau_score"]} Description :{row["summarize"]} """)

TAU Score:0.6 Description : A high-calorie meal may increase the concentration of the drug abemaciclib, according to research published in the British Journal of Oncology. 
TAU Score:-0.8 Description : The World Health Organization (WHO) warns that the consumption of grapefruit juice during treatment with the opioid fentanyl may increase the risk of an adverse reaction to the drug. 
TAU Score:-0.35 Description : AUC (AUC) for docetaxel, a drug used to treat leukaemia, has been reduced to 35% after a trial with grapefruit juice. 
TAU Score:0.7 Description : Ethanol can be used as a treatment for painkiller overdoses. 
TAU Score:0.7 Description :Dextromethorphan and ethanol can be used as a treatment for alcohol addiction. 


In [32]:
fdi_map.drop(columns='tau', inplace= True)

/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_19659/3168122293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdi_map.drop(columns='tau', inplace= True)


In [33]:
fdi_map.iloc[1, 5]

'grapefruit juice'

In [34]:
fdi_map

,texts_ID,food,drug,clean_text,summarize,food_,drug_,relationship,tau_score
0,0,grapefruit,abemaciclib,Moderate Food Interaction||GENERALLY AVOID: G...,A high-calorie meal may increase the concentr...,high-calorie meal,abemaciclib,increasing concentration,0.60
1,2,grapefruit,fentanyl,GENERALLY AVOID: Consumption of grapefruit ju...,The World Health Organization (WHO) warns tha...,grapefruit juice,fentanyl,degrading health,-0.80
2,7,grapefruit,paclitaxel,Moderate Food Interaction||MONITOR: Coadminis...,"AUC (AUC) for docetaxel, a drug used to treat...",grapefruit juice,docetaxel,degrading health,-0.35
3,11,ethanol,opioid analgesics,Moderate Drug Interaction||GENERALLY AVOID: E...,Ethanol can be used as a treatment for painki...,Ethanol,Painkillers,Improving health,0.70
4,13,ethanol,dextromethorphan,Moderate Drug Interaction||GENERALLY AVOID: T...,Dextromethorphan and ethanol can be used as a ...,ethanol,dextromethorphan,improving health,0.70


In [35]:
# content_ = fdi_map.choices[0].message.content

In [36]:
# start_index = content_.find('{')  # Add 1 to exclude the start_char
# end_index = content_.find("}")+1
# json.load(fdi_map.choices[0].message.content[start_index:end_index].replace("""\n""",""))

In [37]:
# json.loads(fdi_map.choices[0].message.content)

In [1]:
%load_ext autoreload
%autoreload 2


In [15]:
from medisure.ml_logic.FDI import get_tau_score

fdi = get_tau_score()

/Users/raynasser/code/raynasser/MediSure/medisure/ml_logic/FDI.py:94: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  


Start Summarization...


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 62, but your input_length is only 23. Since this is a summarization task, where outputs shorter th

KeyboardInterrupt: 

In [10]:
fdi.isnull().sum()

texts_ID      0
food          0
drug          0
clean_text    0
dtype: int64

In [13]:
fdi[fdi["clean_text"]=="nan"]

,texts_ID,food,drug,clean_text


In [20]:
ddi_a = pd.read_csv(os.path.join(ddi, 'A_alimentary_tract_metabolism_drugs.csv'))#, index_col='id')

ddi_a


,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
0,DDInter1263,Naltrexone,DDInter1,Abacavir,Moderate
1,DDInter1,Abacavir,DDInter1348,Orlistat,Moderate
2,DDInter58,Aluminum hydroxide,DDInter582,Dolutegravir,Major
3,DDInter112,Aprepitant,DDInter582,Dolutegravir,Minor
4,DDInter138,Attapulgite,DDInter582,Dolutegravir,Major
...,...,...,...,...,...
56362,DDInter1022,Lansoprazole,DDInter888,Hydrocortisone butyrate,Unknown
56363,DDInter1675,Simvastatin,DDInter888,Hydrocortisone butyrate,Unknown
56364,DDInter1906,Ursodeoxycholic acid,DDInter888,Hydrocortisone butyrate,Unknown
56365,DDInter1318,Nystatin,DDInter1097,Loteprednol etabonate,Unknown


In [21]:
ddi_b = pd.read_csv(os.path.join(ddi, 'B_blood_drugs.csv'))#, index_col='id')

# ddi_b

In [22]:
ddi_d = pd.read_csv(os.path.join(ddi, 'D_dermatologicals_drugs.csv'))#, index_col='id')

# ddi_d

In [23]:
ddi_h = pd.read_csv(os.path.join(ddi, 'H_hormonal_drugs.csv'))#, index_col='id')

# ddi_h

In [24]:
ddi_l = pd.read_csv(os.path.join(ddi, 'L_antineoplastic_immunomodulating_agents_drugs.csv'))#, index_col='id')

# ddi_l

In [25]:
ddi_p = pd.read_csv(os.path.join(ddi, 'P_antiparasitic_insecticides_repellents_drugs.csv'))#, index_col='id')

# ddi_p

In [26]:
ddi_r = pd.read_csv(os.path.join(ddi, 'P_antiparasitic_insecticides_repellents_drugs.csv'))#, index_col='id')

# ddi_r

In [27]:
ddi_v = pd.read_csv(os.path.join(ddi, 'P_antiparasitic_insecticides_repellents_drugs.csv'))#, index_col='id')

# ddi_v

In [28]:

all_ddi = pd.concat([ddi_a, ddi_b, ddi_d, ddi_h, ddi_l, ddi_p, ddi_r, ddi_v], ignore_index=True)
all_ddi


,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
0,DDInter1263,Naltrexone,DDInter1,Abacavir,Moderate
1,DDInter1,Abacavir,DDInter1348,Orlistat,Moderate
2,DDInter58,Aluminum hydroxide,DDInter582,Dolutegravir,Major
3,DDInter112,Aprepitant,DDInter582,Dolutegravir,Minor
4,DDInter138,Attapulgite,DDInter582,Dolutegravir,Major
...,...,...,...,...,...
190775,DDInter1202,Metronidazole,DDInter964,Iopromide,Unknown
190776,DDInter893,Hydroxychloroquine,DDInter1942,Vitamin D,Unknown
190777,DDInter1202,Metronidazole,DDInter801,Galactose,Unknown
190778,DDInter134,Atovaquone,DDInter801,Galactose,Unknown


In [15]:
eda_pipeline(all_ddi)

_________________________________


Columns:
Index(['DDInterID_A', 'Drug_A', 'DDInterID_B', 'Drug_B', 'Level'], dtype='object')


_________________________________


'Total Rows: 190,780'

_________________________________


Unique data types:
[dtype('O')]
_________________________________


Describe the DataFrame (including categorical data):
        DDInterID_A         Drug_A DDInterID_B         Drug_B     Level
count        190780         190780      190780         190780    190780
unique         1738           1738        1833           1833         4
top     DDInter1855  Triamcinolone  DDInter513  Dexamethasone  Moderate
freq           1513           1513        1849           1849    110781
_________________________________


Number of duplicate rows: 49,381
_________________________________


Missing values per column:


,Missing Values
DDInterID_A,0
Drug_A,0
DDInterID_B,0
Drug_B,0
Level,0


,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
0,DDInter1263,Naltrexone,DDInter1,Abacavir,Moderate
1,DDInter1,Abacavir,DDInter1348,Orlistat,Moderate
2,DDInter58,Aluminum hydroxide,DDInter582,Dolutegravir,Major
3,DDInter112,Aprepitant,DDInter582,Dolutegravir,Minor
4,DDInter138,Attapulgite,DDInter582,Dolutegravir,Major
...,...,...,...,...,...
190775,DDInter1202,Metronidazole,DDInter964,Iopromide,Unknown
190776,DDInter893,Hydroxychloroquine,DDInter1942,Vitamin D,Unknown
190777,DDInter1202,Metronidazole,DDInter801,Galactose,Unknown
190778,DDInter134,Atovaquone,DDInter801,Galactose,Unknown


In [39]:
duplicated_rows = all_ddi[all_ddi.duplicated(keep=False)]


In [40]:
duplicated_rows[(duplicated_rows['DDInterID_A'] == 'DDInter134') & (duplicated_rows['DDInterID_B'] == 'DDInter1197')]

,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
8865,DDInter134,Atovaquone,DDInter1197,Metoclopramide,Moderate
175167,DDInter134,Atovaquone,DDInter1197,Metoclopramide,Moderate
180659,DDInter134,Atovaquone,DDInter1197,Metoclopramide,Moderate
186151,DDInter134,Atovaquone,DDInter1197,Metoclopramide,Moderate


In [44]:
all_ddi.drop_duplicates(inplace=True)

In [46]:
np.sum(all_ddi.DDInterID_A == 'DDInter1')

all_ddi[all_ddi.DDInterID_A == 'DDInter1'].duplicated().sum()

# duplicated_rows[(duplicated_rows['DDInterID_A'] == 'DDInter1') & (duplicated_rows['DDInterID_B'] == 'DDInter246')]

0

In [48]:
all_ddi.shape

print(f'Total Rows: {all_ddi.shape[0]:,}')

Total Rows: 141,399


In [55]:
all_ddi = all_ddi.applymap(lower)
all_ddi


,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
0,ddinter1263,naltrexone,ddinter1,abacavir,moderate
1,ddinter1,abacavir,ddinter1348,orlistat,moderate
2,ddinter58,aluminum hydroxide,ddinter582,dolutegravir,major
3,ddinter112,aprepitant,ddinter582,dolutegravir,minor
4,ddinter138,attapulgite,ddinter582,dolutegravir,major
...,...,...,...,...,...
179784,ddinter893,hydroxychloroquine,ddinter511,desvenlafaxine,unknown
179787,ddinter893,hydroxychloroquine,ddinter1097,loteprednol etabonate,unknown
179790,ddinter893,hydroxychloroquine,ddinter19,acetylcysteine,unknown
179792,ddinter893,hydroxychloroquine,ddinter1942,vitamin d,unknown


In [56]:
all_ddi.duplicated().sum()

0

In [57]:
all_ddi.columns

Index(['DDInterID_A', 'Drug_A', 'DDInterID_B', 'Drug_B', 'Level'], dtype='object')

In [88]:
import os
import json
import openai

def ddi_tau_score(drug_a, drug_b, level):
    # Define the API key and endpoint
    API_KEY = os.environ.get("API_KEY")

    # Set up headers for the API request
    client = openai.OpenAI(api_key=API_KEY)

    # Prepare the API call to analyze drug-drug interaction based on level
    prompt = f"""
    You are tasked with determining the interaction mechanism between two drugs.
    The drugs are:
    - Drug A: {drug_a}
    - Drug B: {drug_b}

    The level of interaction is: {level}

    Based on this information, describe the interaction mechanism (inhibition, synergy, antagonism, etc.), and return a tau score that reflects the strength of the interaction. Return the result as a JSON object in the following format:
        {{
            "drug_a": "{drug_a}",
            "drug_b": "{drug_b}",
            "mechanism": "",
            "tau": 0
        }}
    Only return the JSON object.
    """

    try:
        # API call to OpenAI
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        # Log the raw response for debugging
        response_content = completion.choices[0].message.content
        print(f"Response for {drug_a} and {drug_b}: {response_content}")

        # Check if response is empty or invalid
        if not response_content.strip():
            print(f"Empty response for {drug_a} and {drug_b}")
            return {
                "drug_a": drug_a,
                "drug_b": drug_b,
                "mechanism": "unknown",
                "tau": 0
            }

        # Try to load the JSON response
        return json.loads(response_content)

    except (json.JSONDecodeError, KeyError) as e:
        # Handle JSON decode errors or missing keys
        print(f"Error processing response for {drug_a} and {drug_b}: {e}")
        return {
            "drug_a": drug_a,
            "drug_b": drug_b,
            "mechanism": "unknown",
            "tau": 0
        }

    except Exception as e:
        # Catch all other errors (e.g., connection issues)
        print(f"An error occurred: {e}")
        return {
            "drug_a": drug_a,
            "drug_b": drug_b,
            "mechanism": "unknown",
            "tau": 0
        }

    return json.loads(completion.choices[0].message.content)


In [86]:
# import os
# import json
# import openai

# def ddi_tau_score(drug_a, drug_b, level):
#     # Define the API key and endpoint
#     API_KEY = os.environ.get("API_KEY")

#     # Set up headers for the API request
#     client = openai.OpenAI(api_key=API_KEY)

#     # Create the request with structured roles (assistant and user), formatting the drug names
#     try:
#         completion = client.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "assistant", "content": f"""You are tasked with determining the interaction mechanism between two drugs.
#                 The drugs are:
#                 - Drug A: {drug_a}
#                 - Drug B: {drug_b}

#                 The level of interaction is: {level}

#                 Based on this information, describe the interaction mechanism (inhibition, synergy, antagonism, etc.), and return a tau score that reflects the strength of the interaction.
#                 Return the result through a JSON object. The response must be in the following JSON format:
#                 {{
#                     "drug_a": "{drug_a}",
#                     "drug_b": "{drug_b}",
#                     "mechanism": "",
#                     "tau": 0
#                 }}
#                 Return only the JSON object."""},
#                 {"role": "user", "content": f"The interaction level is: {level}"}
#             ],
#             response_format={"type": "json_object"}
#         )

#         # Log the raw response for debugging
#         response_content = completion.choices[0].message.content
#         print(f"Response for {drug_a} and {drug_b}: {response_content}")

#         # Validate that the response is a valid JSON string
#         if response_content.strip().startswith("{") and response_content.strip().endswith("}"):
#             # Try to load the JSON response
#             return json.loads(response_content)
#         else:
#             print(f"Invalid JSON format for {drug_a} and {drug_b}")
#             return {
#                 "drug_a": drug_a,
#                 "drug_b": drug_b,
#                 "mechanism": "unknown",
#                 "tau": 0
#             }

#     except (json.JSONDecodeError, KeyError) as e:
#         # Handle JSON decode errors or missing keys
#         print(f"Error processing response for {drug_a} and {drug_b}: {e}")
#         return {
#             "drug_a": drug_a,
#             "drug_b": drug_b,
#             "mechanism": "unknown",
#             "tau": 0
#         }

#     except Exception as e:
#         # Catch all other errors (e.g., connection issues)
#         print(f"An error occurred: {e}")
#         return {
#             "drug_a": drug_a,
#             "drug_b": drug_b,
#             "mechanism": "unknown",
#             "tau": 0
#         }


In [90]:
all_ddi50 = all_ddi.head(7)
all_ddi50

,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
0,ddinter1263,naltrexone,ddinter1,abacavir,moderate
1,ddinter1,abacavir,ddinter1348,orlistat,moderate
2,ddinter58,aluminum hydroxide,ddinter582,dolutegravir,major
3,ddinter112,aprepitant,ddinter582,dolutegravir,minor
4,ddinter138,attapulgite,ddinter582,dolutegravir,major
5,ddinter270,calcium acetate,ddinter582,dolutegravir,major
6,ddinter271,calcium carbonate,ddinter582,dolutegravir,major


In [89]:
# Select a specific row by index (for example, the first row)
single_row = all_ddi.iloc[3]

# Extract the values for Drug_A, Drug_B, and Level from the row
drug_a = single_row['Drug_A']
drug_b = single_row['Drug_B']
level = single_row['Level']

# Call the tau_score function for just this one row
result = ddi_tau_score(drug_a, drug_b, level)

# Print the result
print(result)


Response for aprepitant and dolutegravir: ```json
{
    "drug_a": "aprepitant",
    "drug_b": "dolutegravir",
    "mechanism": "inhibition",
    "tau": 0.2
}
```
Error processing response for aprepitant and dolutegravir: Expecting value: line 1 column 1 (char 0)
{'drug_a': 'aprepitant', 'drug_b': 'dolutegravir', 'mechanism': 'unknown', 'tau': 0}


In [71]:
all_ddi.Level.value_counts()

moderate    84015
unknown     26716
major       24190
minor        6478
Name: Level, dtype: int64

In [68]:
# all_ddi50['Interaction_Details'] = all_ddi.apply(
#         lambda row: ddi_tau_score(row['Drug_A'], row['Drug_B'], row['Level']),
#         axis=1
#     )


all_ddi50['Interaction_Details'] = all_ddi50.apply(
    lambda row: ddi_tau_score(row['Drug_A'],
                              row['Drug_B'],
                              row['Level']),
    axis=1)


Response for naltrexone and abacavir: ```json
{
    "drug_a": "naltrexone",
    "drug_b": "abacavir",
    "mechanism": "antagonism",
    "tau": 0.3
}
```
Error processing response for naltrexone and abacavir: Expecting value: line 1 column 1 (char 0)
Response for abacavir and orlistat: ```json
{
    "drug_a": "abacavir",
    "drug_b": "orlistat",
    "mechanism": "inhibition",
    "tau": 0.4
}
```
Error processing response for abacavir and orlistat: Expecting value: line 1 column 1 (char 0)
Response for aluminum hydroxide and dolutegravir: ```json
{
    "drug_a": "aluminum hydroxide",
    "drug_b": "dolutegravir",
    "mechanism": "chelation",
    "tau": 0.8
}
```
Error processing response for aluminum hydroxide and dolutegravir: Expecting value: line 1 column 1 (char 0)
Response for aprepitant and dolutegravir: ```json
{
    "drug_a": "aprepitant",
    "drug_b": "dolutegravir",
    "mechanism": "inhibition",
    "tau": 0.2
}
```
Error processing response for aprepitant and dolutegravi

KeyboardInterrupt: 

In [72]:
all_ddi['Tau_Score'] = all_ddi['Interaction_Details'].apply(lambda x: x['tau'])


KeyError: 'Interaction_Details'

In [77]:
# import os
# import json
# import openai
# import time

# def drug_interaction_effects(drug_a, drug_b, retries=3):
#     # Define the API key and endpoint
#     API_KEY = os.environ.get("API_KEY")

#     # Set up headers for the API request
#     client = openai.OpenAI(api_key=API_KEY)

#     prompt = f"""
#     You are tasked with analyzing the effects of two drugs when taken together. The drugs are:
#     - Drug A: {drug_a}
#     - Drug B: {drug_b}

#     Describe the potential effects when these two drugs are co-administered, including any side effects, possible increased toxicity, synergistic effects, or contraindications. Return the result as a JSON object in the following format:
#         {{
#             "drug_a": "{drug_a}",
#             "drug_b": "{drug_b}",
#             "effects": [
#                 {{
#                     "effect": "",
#                     "severity": "",
#                     "details": ""
#                 }}
#             ]
#         }}
#     Only return the JSON object.
#     """

#     for attempt in range(retries):
#         try:
#             # API call to OpenAI
#             completion = client.chat.completions.create(
#                 model="gpt-4o",
#                 messages=[
#                     {"role": "user", "content": prompt}
#                 ]
#             )

#             # Log the raw response for debugging
#             response_content = completion.choices[0].message.content
#             print(f"Raw response for {drug_a} and {drug_b}: {response_content}")

#             # Check if response is empty or invalid
#             if not response_content.strip():
#                 print(f"Empty response for {drug_a} and {drug_b}, retrying...")
#                 continue  # Retry the request

#             # Try to load the JSON response
#             return json.loads(response_content)

#         except (json.JSONDecodeError, KeyError) as e:
#             print(f"Error processing response for {drug_a} and {drug_b}: {e}")
#             continue  # Retry the request

#         except Exception as e:
#             print(f"An error occurred: {e}")
#             time.sleep(1)  # Add a delay before retrying

#     # After retrying, if all attempts fail, return a default response
#     return {
#         "drug_a": drug_a,
#         "drug_b": drug_b,
#         "effects": [
#             {
#                 "effect": "unknown",
#                 "severity": "unknown",
#                 "details": "Unable to retrieve interaction details after retries."
#             }
#         ]
#     }


In [79]:
# # Example interaction between naltrexone and abacavir
# result = drug_interaction_effects("naltrexone", "abacavir")

# # Output the result
# print(json.dumps(result, indent=4))
